In [129]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from xgboost import XGBRegressor

from data_processing import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from data_transformar import *
import lightgbm as lgb
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor

warnings.simplefilter("ignore")
% matplotlib inline
% load_ext autoreload
% autoreload 2

sns.set(style="darkgrid")

UsageError: Line magic function `%` not found.


In [92]:
from sklearn.metrics import mean_squared_error

answers = pd.read_csv("./result-with-best.csv")


def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


def answers_score(model):
    print("RMSLE sub: " + str(rmsle(model.predict(validation), np.log1p(answers["SalePrice"]))))


def evaluate(model, X, y):
    preds = model.predict(X)
    print("RMSLE: " + str(rmsle(preds, y)))


def to_categorical(X):
    for c in X.columns:
        col_type = X[c].dtype
        if col_type == 'object' or col_type.name == 'category':
            X[c] = X[c].astype('category')

In [144]:
data = pd.read_csv("./train.csv")
data = data.drop(columns=["Id"])
#data = remove_outliers(data)

In [145]:
y = np.log1p(data["SalePrice"])
X = data.drop(columns=["SalePrice"])

In [146]:
transformer = DataTransformer()
X_encoded = transformer.fit_transform(X, True)
X_cat = transformer.fit_transform(X, False)
to_categorical(X_cat)
cat_features = list(X_cat.select_dtypes(include=['category']).columns)

validation = pd.read_csv("./test.csv")
val_ids = validation["Id"]
validation = validation.drop(columns=["Id"])

val_encoded = transformer.fit_transform(validation, True)
val_cat = transformer.fit_transform(validation, False)
to_categorical(val_cat)

In [147]:
xgb_params = {'lambda': 11.946656615633028,
              'learning_rate': 0.002119415669803155,
              'max_depth': 863,
              'n_estimators': 30000,
              'subsample': 0.1382402507540342}

xgb_model = XGBRegressor(**xgb_params, seed=0)
xgb_model.fit(X_encoded, y)
evaluate(xgb_model, val_encoded, np.log1p(answers['SalePrice']))

RMSLE: 0.1228518431429536


In [148]:
lgbm_params = {'colsample_bytree': 0.17244191446833482, 'learning_rate': 0.010783153461441895, 'max_depth': 26,
               'n_estimators': 3347, 'num_leaves': 28, 'reg_alpha': 0.6690324820643466,
               'reg_lambda': 0.013450189775519253, 'subsample': 0.6868741372881852,
               'num_iterations': 2000}
fit_params = {
    "eval_metric": 'rmse',
    'verbose': -1,
    'categorical_feature': cat_features
}

lgbm_model = lgb.LGBMRegressor(**lgbm_params)
lgbm_model.fit(X_cat, y, **fit_params)
evaluate(lgbm_model, X_cat, y)
evaluate(lgbm_model, val_cat, np.log1p(answers['SalePrice']))

RMSLE: 0.060069088919828026
RMSLE: 0.12435667527220387


In [142]:
categorical_features = np.where(X_cat.loc[:, X_cat.columns.values].dtypes == "category")[0]
train_pool = Pool(X_cat, y, cat_features=categorical_features)

cat_model = CatBoostRegressor(iterations=400, task_type="GPU",  devices='0:1')
cat_model.fit(train_pool, verbose=0, plot=False)

evaluate(cat_model, X_cat, y)
evaluate(cat_model, val_cat, np.log1p(answers['SalePrice']))

RMSLE: 0.06943040829366837
RMSLE: 0.1261692980946463


In [149]:
blended_res = 1/3*cat_model.predict(val_cat) + 1/3*lgbm_model.predict(val_cat) + 1/3*xgb_model.predict(val_encoded)
print("RMSLE: " + str(rmsle(blended_res, np.log1p(answers['SalePrice']))))

RMSLE: 0.11959896005429083


In [134]:
save_res(blended_res)